In [ ]:
import pygame as pg
import os 
import time 
import utils as ut

# Standard 
BLUE = (0,80,239)
BLACK = (0,0,0)
GREEN = (0,138,0)
RED = (229,20,0)
WHITE = (255,255,255)

# Initializing
pg.init()

# Screen parameters
clock = pg.time.Clock()
screen = pg.display.set_mode((1600,900),pg.NOFRAME|pg.SCALED)
pg.display.set_caption("Chapelié")

# Importing all images/fonts
[flag_be, flag_fr, flag_rma, flag_x] = ut.prepare_flags()
boat = pg.transform.scale(pg.image.load(os.path.join(".", "figures","warship.png")),(375,375))
background_image = pg.transform.scale(pg.image.load(os.path.join(".", "figures","canal.png")),(4000,2000)).convert()
background_image.set_alpha(128)
[font_30, font_50, font_70] = ut.import_fonts()

# Assigning all utilites
colorswapper = ut.ColorSwapper([WHITE,GREEN],.2,10)
countdown = ut.Countdown(screen,time.time(),30,font_50,WHITE)
distance_marker1 = ut.DistanceMarkers(screen,font_30,200,125)
distance_marker2 = ut.DistanceMarkers(screen,font_30,200,-125)
background = ut.Background(screen,background_image,4)
boat_rma = ut.Boat(screen,boat,flag_be,175)
boat_x = ut.Boat(screen,boat,flag_be,-75)
flag_rma = ut.AnimatedFlag(screen,flag_rma,(-100-flag_rma[0].get_width(),flag_rma[0].get_height()//2))
flag_x = ut.AnimatedFlag(screen,flag_x,(100,flag_x[0].get_height()//2))

count = 0
start = time.time()
for i in range(100):
    # Time
    duration = time.time()-start
    
    # Swapping colors
    distance_marker1.color = colorswapper.swap(count,200,duration)
    
    # Rendering of elements
    background.blit(count)
    distance_marker1.blit(count)
    distance_marker2.blit(count)
    background.blit(count)
    flag_rma.blit()
    flag_x.blit()
    ut.render_text(screen,font_30,"VS",(screen.get_width()//2,screen.get_height()//2))
    boat_rma.blit(count)
    boat_x.blit(count)
    countdown.blit()

    # Pygame stuff
    clock.tick(30)
    pg.display.flip()
    screen.fill(BLACK)
    count += 1

pg.display.quit()
pg.quit()

In [ ]:
#TODO: Can be implemented
score1 = font_50.render("DISTANCE: "+str(count) + " m", True, color)
score1_rect = score1.get_rect()
score1_rect.midtop = (screen.get_width()//2,100)
score2 = font_50.render("DISTANCE: "+str(count) + " m", True, WHITE)
score2_rect = score2.get_rect()
score2_rect.midbottom = (screen.get_width()//2,screen.get_height()-100)
    

In [ ]:
from pyrow import pyrow as pr
from threading import Thread

In [ ]:
devices = [pr.PyErg(erg) for erg in list(pr.find())]
threads = []
for device in devices:
    threads.append(Thread(target=device.get_monitor))

In [ ]:
def monitor_stream(device:pr.PyErg):
    while True:
        device.get_monitor()